In [1]:
import json
import os

# Path to the data folder
data_folder = 'data'

# Load the first JSON file
file_path = os.path.join(data_folder, 'mpd.slice.0-999.json')

with open(file_path, 'r') as f:
    data = json.load(f)

# Explore the structure of the data
print("Keys in the JSON file:", data.keys())
print("\nSample Playlist Metadata:", data['playlists'][0])
print("\nSample Tracks in a Playlist:", data['playlists'][0]['tracks'][:5])  # First 5 tracks


Keys in the JSON file: dict_keys(['info', 'playlists'])

Sample Playlist Metadata: {'name': 'Throwbacks', 'collaborative': 'false', 'pid': 0, 'modified_at': 1493424000, 'num_tracks': 52, 'num_albums': 47, 'num_followers': 1, 'tracks': [{'pos': 0, 'artist_name': 'Missy Elliott', 'track_uri': 'spotify:track:0UaMYEvWZi0ZqiDOoHU3YI', 'artist_uri': 'spotify:artist:2wIVse2owClT7go1WT98tk', 'track_name': 'Lose Control (feat. Ciara & Fat Man Scoop)', 'album_uri': 'spotify:album:6vV5UrXcfyQD1wu4Qo2I9K', 'duration_ms': 226863, 'album_name': 'The Cookbook'}, {'pos': 1, 'artist_name': 'Britney Spears', 'track_uri': 'spotify:track:6I9VzXrHxO9rA9A5euc8Ak', 'artist_uri': 'spotify:artist:26dSoYclwsYLMAKD3tpOr4', 'track_name': 'Toxic', 'album_uri': 'spotify:album:0z7pVBGOD7HCIB7S8eLkLI', 'duration_ms': 198800, 'album_name': 'In The Zone'}, {'pos': 2, 'artist_name': 'Beyoncé', 'track_uri': 'spotify:track:0WqIKmW4BTrj3eJFmnCKMv', 'artist_uri': 'spotify:artist:6vWDO969PvNqNYHIOW5v0m', 'track_name': 'Crazy

In [7]:
import os
import json
import pandas as pd

# Path to the data folder
data_folder = 'data'

# Get the first 100 files
files = sorted(os.listdir(data_folder))[:50]

# Define output CSV file
output_file = 'processed_first_50_files.csv'

# Initialize CSV with headers
columns = [
    'playlist_id', 'playlist_name', 'track_position',
    'track_name', 'artist_name', 'album_name', 'duration_ms', 'track_uri'
]
pd.DataFrame(columns=columns).to_csv(output_file, index=False)

# Function to process and append a single file
def process_and_save(file_path, output_file):
    with open(file_path, 'r') as f:
        data = json.load(f)
        rows = []
        for playlist in data['playlists']:
            for track in playlist['tracks']:
                rows.append({
                    'playlist_id': playlist['pid'],
                    'playlist_name': playlist['name'],
                    'track_position': track['pos'],
                    'track_name': track['track_name'],
                    'artist_name': track['artist_name'],
                    'album_name': track['album_name'],
                    'duration_ms': track['duration_ms'],
                    'track_uri': track['track_uri']
                })
    # Convert to a DataFrame
    df = pd.DataFrame(rows)
    # Append to CSV file
    df.to_csv(output_file, mode='a', index=False, header=False)

# Process each file and append to the output CSV
for file_name in files:
    file_path = os.path.join(data_folder, file_name)
    print(f"Processing file: {file_name}")
    process_and_save(file_path, output_file)

# Display a sample of the final output
print(pd.read_csv(output_file).head())


Processing file: mpd.slice.0-999.json
Processing file: mpd.slice.1000-1999.json
Processing file: mpd.slice.10000-10999.json
Processing file: mpd.slice.100000-100999.json
Processing file: mpd.slice.101000-101999.json
Processing file: mpd.slice.102000-102999.json
Processing file: mpd.slice.103000-103999.json
Processing file: mpd.slice.104000-104999.json
Processing file: mpd.slice.105000-105999.json
Processing file: mpd.slice.106000-106999.json
Processing file: mpd.slice.107000-107999.json
Processing file: mpd.slice.108000-108999.json
Processing file: mpd.slice.109000-109999.json
Processing file: mpd.slice.11000-11999.json
Processing file: mpd.slice.110000-110999.json
Processing file: mpd.slice.111000-111999.json
Processing file: mpd.slice.112000-112999.json
Processing file: mpd.slice.113000-113999.json
Processing file: mpd.slice.114000-114999.json
Processing file: mpd.slice.115000-115999.json
Processing file: mpd.slice.116000-116999.json
Processing file: mpd.slice.117000-117999.json
Proc

In [3]:
import pandas as pd

# Load the processed data from the CSV file
final_df = pd.read_csv('processed_first_200_files.csv')

# Display a sample to verify
print(final_df.head())


   playlist_id playlist_name  track_position  \
0            0    Throwbacks               0   
1            0    Throwbacks               1   
2            0    Throwbacks               2   
3            0    Throwbacks               3   
4            0    Throwbacks               4   

                                   track_name        artist_name  \
0  Lose Control (feat. Ciara & Fat Man Scoop)      Missy Elliott   
1                                       Toxic     Britney Spears   
2                               Crazy In Love            Beyoncé   
3                              Rock Your Body  Justin Timberlake   
4                                It Wasn't Me             Shaggy   

                                     album_name  duration_ms  \
0                                  The Cookbook       226863   
1                                   In The Zone       198800   
2  Dangerously In Love (Alben für die Ewigkeit)       235933   
3                                     Justifie

Word2vec

In [4]:
from gensim.models import Word2Vec

# Prepare corpus of track and artist names
corpus = (final_df['track_name'].dropna().tolist() +
          final_df['artist_name'].dropna().tolist())

# Train word2vec
corpus = [item.split() for item in corpus]  # Split names into words
w2v_model = Word2Vec(sentences=corpus, vector_size=100, window=5, min_count=1, workers=4)

# Example: Get embedding for a track
track_vector = w2v_model.wv['Lose']  # Example: 'Lose' from "Lose Yourself"


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

Item2vec

In [2]:
from gensim.models import Word2Vec

# Treat each playlist as a sentence and each track URI as a word
playlists = final_df.groupby('playlist_id')['track_uri'].apply(list).tolist()

# Train Item2Vec model
item2vec_model = Word2Vec(sentences=playlists, vector_size=100, window=5, min_count=1, workers=4)

# Get vector for a track
track_vector = item2vec_model.wv['spotify:track:0UaMYEvWZi0ZqiDOoHU3YI']


FISM

In [ ]:
import numpy as np
from sklearn.decomposition import TruncatedSVD

# Create user-item interaction matrix
interaction_matrix = pd.crosstab(final_df['playlist_id'], final_df['track_uri'])

# Apply SVD
svd = TruncatedSVD(n_components=100)
item_embeddings = svd.fit_transform(interaction_matrix.T)

# Example: Get embedding for a specific track
track_index = interaction_matrix.columns.get_loc('spotify:track:0UaMYEvWZi0ZqiDOoHU3YI')
track_vector = item_embeddings[track_index]


Pursue with TF-IDF with only playlist name

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Extract playlist names
playlist_titles = final_df['playlist_name'].fillna('')  # Handle missing titles

# Create TF-IDF vectors
tfidf_vectorizer = TfidfVectorizer(max_features=500)
tfidf_matrix = tfidf_vectorizer.fit_transform(playlist_titles)


In [6]:
import pandas as pd
import dask.array as da
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load the dataset
final_df = pd.read_csv('processed_first_100_files.csv')

# Group by playlist_id and aggregate playlist titles (or combine track metadata if needed)
playlist_level_df = (
    final_df.groupby('playlist_id')
    .agg({'playlist_name': 'first'})  # Use the playlist title
    .reset_index()
)

# Create a TF-IDF matrix for playlist titles
tfidf_vectorizer = TfidfVectorizer(max_features=500)  # Limit to top 500 features for performance
tfidf_matrix = tfidf_vectorizer.fit_transform(playlist_level_df['playlist_name'].fillna(''))

# Use the sparse TF-IDF matrix directly
tfidf_sparse = tfidf_matrix  # Keep as sparse (don't call .toarray())

# Define chunk size for Dask processing
chunk_size = 1000

# Wrap the sparse matrix with Dask for chunking
tfidf_dask = da.from_array(tfidf_sparse, chunks=(chunk_size, tfidf_sparse.shape[1]))

# Function to compute pairwise cosine similarity for a chunk
def compute_cosine_similarity_sparse(chunk, tfidf_sparse):
    """Compute cosine similarity for the chunk against the full sparse matrix."""
    return cosine_similarity(chunk, tfidf_sparse, dense_output=False)  # Keep as sparse

# Apply the cosine similarity function on chunks
similarity_results = tfidf_dask.map_blocks(
    compute_cosine_similarity_sparse,
    tfidf_sparse,
    dtype=np.float64
)

# Compute similarity incrementally for a specific playlist
playlist_index = 31  # Example: Compute similarity for the first playlist
similarities = similarity_results[playlist_index].compute()

# Get top 5 similar playlists by similarity score
top_indices = np.argsort(similarities.toarray().flatten())[::-1][1:101]  # Exclude self (index 0)

# Map indices back to playlist_id
top_playlist_ids = playlist_level_df.iloc[top_indices]['playlist_id'].tolist()

# Output the top playlist IDs
print("Top 5 similar playlist IDs:", top_playlist_ids)


Top 5 similar playlist IDs: [16538, 148846, 150551, 186074, 183738, 139314, 148090, 149260, 152436, 146505, 181906, 166315, 147263, 179156, 152230, 144643, 170511, 171060, 182791, 139918, 176039, 16181, 159295, 167282, 186862, 149908, 156669, 187273, 172482, 162576, 15609, 179647, 152361, 182961, 180841, 172478, 148839, 165381, 153057, 156665, 167147, 149906, 170633, 146768, 166418, 148738, 151706, 15595, 183134, 185236, 182228, 183132, 165824, 162777, 144714, 169951, 181869, 14748, 172758, 14734, 139810, 140645, 175368, 187237, 14562, 146608, 152150, 171169, 184250, 170212, 152974, 148831, 153337, 165697, 188924, 157704, 147633, 182385, 162349, 169674, 169936, 13805, 175035, 179608, 173952, 169542, 140677, 143494, 13424, 143170, 160036, 148352, 139916, 12982, 11929, 151189, 146603, 171430, 143408, 165797]


In [6]:
# Filter the DataFrame for the given playlist IDs and retrieve titles
matching_playlists = playlist_level_df[playlist_level_df['playlist_id'].isin(top_playlist_ids)]

# Display the playlist IDs and their corresponding titles
for _, row in matching_playlists.iterrows():
    print(f"Playlist ID: {row['playlist_id']}, Title: {row['playlist_name']}")


Playlist ID: 11929, Title: Running
Playlist ID: 12982, Title: running 
Playlist ID: 13424, Title: Running
Playlist ID: 13805, Title: Running
Playlist ID: 14562, Title: running
Playlist ID: 14734, Title: Running 
Playlist ID: 14748, Title: running 
Playlist ID: 15595, Title: running
Playlist ID: 15609, Title: Running
Playlist ID: 16181, Title: Running 2.0
Playlist ID: 16538, Title: Running
Playlist ID: 139314, Title: Running
Playlist ID: 139810, Title: Running 3
Playlist ID: 139916, Title: running
Playlist ID: 139918, Title: running
Playlist ID: 140645, Title: running
Playlist ID: 140677, Title: running 2
Playlist ID: 143170, Title: running
Playlist ID: 143408, Title: running
Playlist ID: 143494, Title: Running
Playlist ID: 144643, Title: Running
Playlist ID: 144714, Title: running
Playlist ID: 146505, Title: Running
Playlist ID: 146603, Title: Running
Playlist ID: 146608, Title: Running
Playlist ID: 146768, Title: running 2.0
Playlist ID: 147263, Title: running
Playlist ID: 147633, Tit

In [7]:
# Get the tracks in the target playlist
target_tracks = set(final_df[final_df['playlist_id'] == playlist_index]['track_name'])

# Aggregate tracks from the specified similar playlists
similar_tracks = final_df[final_df['playlist_id'].isin(top_playlist_ids)]['track_name']

# Exclude tracks already in the target playlist
filtered_tracks = similar_tracks[~similar_tracks.isin(target_tracks)]

# Rank tracks by frequency of occurrence
track_recommendations = filtered_tracks.value_counts().head(10)

# Print recommended tracks and their frequencies
print(f"Recommended Tracks for Playlist ID {playlist_index} (not in original):")
for track_name, freq in track_recommendations.items():
    print(f"Track Name: {track_name}, Frequency: {freq}")


Recommended Tracks for Playlist ID 31 (not in original):
Track Name: Can't Hold Us - feat. Ray Dalton, Frequency: 14
Track Name: Don't Let Me Down, Frequency: 13
Track Name: GDFR (feat. Sage The Gemini & Lookas), Frequency: 11
Track Name: Hey Mama (feat. Nicki Minaj, Bebe Rexha & Afrojack), Frequency: 10
Track Name: Closer, Frequency: 10
Track Name: Turn Down for What, Frequency: 10
Track Name: Centuries, Frequency: 9
Track Name: Wake Me Up - Radio Edit, Frequency: 9
Track Name: Forever, Frequency: 9
Track Name: Ni**as In Paris, Frequency: 9


Only focus on artist name

In [16]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load the dataset
final_df = pd.read_csv('processed_first_100_files.csv')

# Step 1: Aggregate artist names for each playlist
artist_aggregated = (
    final_df.groupby('playlist_id')['artist_name']
    .apply(lambda x: ' '.join(x.dropna().astype(str)))  # Combine all artist names
    .reset_index()
    .rename(columns={'artist_name': 'artist_features'})  # Rename for clarity
)

# Step 2: Create a TF-IDF matrix for artist names
tfidf_vectorizer = TfidfVectorizer(max_features=500)  # Limit features for performance
artist_tfidf = tfidf_vectorizer.fit_transform(artist_aggregated['artist_features'])

# Step 3: Define a function to compute artist similarity
def compute_artist_similarity(index, artist_tfidf):
    """Compute similarity for a given playlist index based on artist features."""
    similarity = cosine_similarity(artist_tfidf[index], artist_tfidf).flatten()
    return similarity

# Step 4: Compute similarity for playlist 0
target_playlist_id = 31
target_playlist_index = artist_aggregated[artist_aggregated['playlist_id'] == target_playlist_id].index[0]
artist_similarity = compute_artist_similarity(target_playlist_index, artist_tfidf)

# Step 5: Get the top 5 most similar playlists by artist overlap
top_indices = np.argsort(artist_similarity)[::-1][1:6]  # Exclude self (index 0)
top_playlist_ids = artist_aggregated.iloc[top_indices]['playlist_id'].tolist()
top_similarity_scores = artist_similarity[top_indices]

# Step 6: Output the top 5 similar playlists and their similarity scores
print("Top 5 similar playlists based on artist names:")
for playlist_id, score in zip(top_playlist_ids, top_similarity_scores):
    print(f"Playlist ID: {playlist_id}, Similarity Score: {score:.4f}")

# Step 7: Recommend tracks not in target playlist but in the top 5 similar playlists
# Get tracks from the target playlist
target_tracks = set(final_df[final_df['playlist_id'] == target_playlist_id]['track_name'])

# Aggregate tracks from the top similar playlists
similar_tracks_df = final_df[final_df['playlist_id'].isin(top_playlist_ids)]
similar_tracks = similar_tracks_df['track_name']

# Exclude tracks already in the target playlist
filtered_tracks = similar_tracks[~similar_tracks.isin(target_tracks)]

# Rank tracks by frequency of occurrence
track_recommendations = filtered_tracks.value_counts()

# Output the recommended tracks
print(f"\nRecommended Tracks for Playlist ID {target_playlist_id} (not in original):")
for track_name, freq in track_recommendations.items():
    print(f"Track Name: {track_name}, Frequency: {freq}")


Top 5 similar playlists based on artist names:
Playlist ID: 183508, Similarity Score: 0.5459
Playlist ID: 950, Similarity Score: 0.5388
Playlist ID: 157989, Similarity Score: 0.5337
Playlist ID: 109133, Similarity Score: 0.5333
Playlist ID: 129405, Similarity Score: 0.5332

Recommended Tracks for Playlist ID 31 (not in original):
Track Name: Firestone, Frequency: 4
Track Name: Love Me Like You Do - From "Fifty Shades Of Grey", Frequency: 3
Track Name: Hands To Myself, Frequency: 3
Track Name: Stay, Frequency: 3
Track Name: Sugar, Frequency: 2
Track Name: Worse Things Than Love, Frequency: 2
Track Name: Middle, Frequency: 2
Track Name: Don't Let Me Down, Frequency: 2
Track Name: Into You, Frequency: 2
Track Name: Can't Feel My Face, Frequency: 2
Track Name: Mary, Frequency: 2
Track Name: Bring Back the Summer (feat. OLY), Frequency: 2
Track Name: Ghost, Frequency: 2
Track Name: What Do You Mean?, Frequency: 2
Track Name: Faded, Frequency: 2
Track Name: Cool for the Summer, Frequency: 2


Pursue with TF-IDF with  playlist name and artist name

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Step 1: Ensure 'artist_features' is created
if 'artist_features' not in locals():
    # Aggregate artist names for each playlist to create 'artist_features'
    artist_aggregated = (
        final_df.groupby('playlist_id')['artist_name']
        .apply(lambda x: ' '.join(x.dropna().unique()))  # Combine unique artist names
        .reset_index()
        .rename(columns={'artist_name': 'artist_features'})  # Rename for clarity
    )

    # Merge 'artist_features' into the playlist-level DataFrame
    playlist_level_df = playlist_level_df.merge(artist_aggregated, on='playlist_id', how='left')

# Step 2: Verify the existence and content of 'artist_features'
print("Sample 'artist_features':")
print(playlist_level_df[['playlist_id', 'artist_features']].head())

# Step 3: Compute TF-IDF Matrices
vectorizer = TfidfVectorizer(max_features=500)  # Limit to top 500 features
name_tfidf = vectorizer.fit_transform(playlist_level_df['playlist_name'].fillna(''))
artist_tfidf = vectorizer.fit_transform(playlist_level_df['artist_features'].fillna(''))

# Step 4: Compute Weighted Similarity
def compute_combined_similarity(index, name_tfidf, artist_tfidf, name_weight, artist_weight):
    """Compute combined similarity for a given playlist index."""
    # Playlist name similarity
    name_similarity = cosine_similarity(name_tfidf[index], name_tfidf).flatten()
    # Artist name similarity
    artist_similarity = cosine_similarity(artist_tfidf[index], artist_tfidf).flatten()
    # Weighted combination
    combined_similarity = (name_weight * name_similarity) + (artist_weight * artist_similarity)
    return combined_similarity

# Step 5: Define Weights and Compute Similarity
name_weight = 0.5
artist_weight = 0.5
playlist_index = 31  # Example: Compute for playlist with ID 31

combined_similarity = compute_combined_similarity(
    playlist_index, name_tfidf, artist_tfidf, name_weight, artist_weight
)

# Step 6: Get Top Similar Playlists
top_indices = np.argsort(combined_similarity)[::-1][1:6]  # Exclude self (index 0)
top_playlist_ids = playlist_level_df.iloc[top_indices]['playlist_id'].tolist()

# Step 7: Output Results
print("Top 5 similar playlist IDs:", top_playlist_ids)

# Step 8: Display Playlist Titles
matching_playlists = playlist_level_df[playlist_level_df['playlist_id'].isin(top_playlist_ids)]
for _, row in matching_playlists.iterrows():
    print(f"Playlist ID: {row['playlist_id']}, Title: {row['playlist_name']}")

# Step 9: Recommend Tracks Not in Target Playlist
# Get tracks from the target playlist
target_tracks = set(final_df[final_df['playlist_id'] == playlist_index]['track_name'])

# Aggregate tracks from the top similar playlists
similar_tracks = final_df[final_df['playlist_id'].isin(top_playlist_ids)]['track_name']

# Filter tracks not in the target playlist
filtered_tracks = similar_tracks[~similar_tracks.isin(target_tracks)]

# Rank by frequency
track_recommendations = filtered_tracks.value_counts().head(10)

# Output recommended tracks
print(f"Recommended Tracks for Playlist ID {playlist_index} (not in original):")
for track_name, freq in track_recommendations.items():
    print(f"Track Name: {track_name}, Frequency: {freq}")


Sample 'artist_features':
   playlist_id                                    artist_features
0            0  Missy Elliott Britney Spears Beyoncé Justin Ti...
1            1  Survivor Daniel Tidwell Kaleptik Ben Foster Le...
2            2  Hoody Loco Park Kyung BTS Lovelyz LEE HI Ailee...
3            3  Camille Saint-Saëns No Vacation Banes World PW...
4            4  The Smashing Pumpkins Oasis Aerosmith Natalie ...
Top 5 similar playlist IDs: [104510, 180416, 147263, 146754, 135241]
Playlist ID: 104510, Title: Running
Playlist ID: 135241, Title: Running 
Playlist ID: 146754, Title: Running
Playlist ID: 147263, Title: running
Playlist ID: 180416, Title: running
Recommended Tracks for Playlist ID 31 (not in original):
Track Name: What Do You Mean?, Frequency: 3
Track Name: Worth It, Frequency: 3
Track Name: Temperature, Frequency: 2
Track Name: Black Widow, Frequency: 2
Track Name: Let Me Hold You (Turn Me On), Frequency: 2
Track Name: Toothbrush, Frequency: 2
Track Name: Desire - Gry

In [6]:
pip install spacy

     ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
     ---------------------------------------- 1.3/1.3 MB 8.4 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencies: still running...
  Installing build dependencies: still running...
  Installing build dependencies: still running...
  Installing build dependencies: still running...
  Installing build dependencies: still running...
  Installing build dependencies: still running...
  Installing build dependencies: still running...
  Installing build dependencies: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> [157 lines of output]
      Ignoring numpy: markers 'python_version < "3.9"' don't match your environment
        Using cached setuptools-75.6.0-py3-none-any.whl.metadata (6.7 kB)
        Installing build dependencies: started
        Installing build dependencies: finished with status 'done'
        Getting requirements to build wheel: started
        Getting requirements to build wheel: finished with status 'done'
        Preparing metadata (pyproject.toml): started
        Preparing metadata (pyproject.toml): finished with status 'done'
        Using cached murmurhash-1.0.11-cp313-cp313-win_amd64.whl.metadata (2.0 kB)
        Installing build dependencies: started
        Installing build dependencies: still running...
        Installing build dependencies: still running...
        Installing build dependencies: still running...
        Installin

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy

# Load spaCy's pre-trained language model
nlp = spacy.load("en_core_web_md")  # Ensure "en_core_web_md" is installed

# Step 1: Expand playlist names using spaCy's semantic similarity
def expand_playlist_name(playlist_name, all_playlist_names, threshold=0.75):
    """
    Expand the playlist name with semantically similar terms from other playlist names.
    
    Parameters:
        playlist_name (str): The name of the current playlist.
        all_playlist_names (iterable): All playlist names in the dataset.
        threshold (float): Minimum similarity score for including related terms.
    
    Returns:
        str: Expanded playlist name with similar terms included.
    """
    expanded_terms = set(playlist_name.split())  # Start with the original words
    target_doc = nlp(playlist_name)  # Convert playlist name to a spaCy Doc
    
    for name in all_playlist_names:
        comparison_doc = nlp(name)  # Convert other playlist names to Docs
        similarity = target_doc.similarity(comparison_doc)  # Calculate similarity
        if similarity > threshold:
            expanded_terms.update(name.split())  # Add related terms
    
    return ' '.join(expanded_terms)

# Apply the function to expand all playlist names
playlist_level_df['expanded_playlist_name'] = playlist_level_df['playlist_name'].apply(
    lambda x: expand_playlist_name(x, playlist_level_df['playlist_name'], threshold=0.75)
)

# Step 2: Ensure 'artist_features' is created
if 'artist_features' not in locals():
    # Aggregate artist names for each playlist to create 'artist_features'
    artist_aggregated = (
        final_df.groupby('playlist_id')['artist_name']
        .apply(lambda x: ' '.join(x.dropna().unique()))  # Combine unique artist names
        .reset_index()
        .rename(columns={'artist_name': 'artist_features'})  # Rename for clarity
    )

    # Merge 'artist_features' into the playlist-level DataFrame
    playlist_level_df = playlist_level_df.merge(artist_aggregated, on='playlist_id', how='left')

# Step 3: Verify the existence and content of 'artist_features'
print("Sample 'artist_features':")
print(playlist_level_df[['playlist_id', 'artist_features']].head())

# Step 4: Compute TF-IDF Matrices
vectorizer = TfidfVectorizer(max_features=500)  # Limit to top 500 features
name_tfidf = vectorizer.fit_transform(playlist_level_df['expanded_playlist_name'].fillna(''))  # Use expanded names
artist_tfidf = vectorizer.fit_transform(playlist_level_df['artist_features'].fillna(''))

# Step 5: Compute Weighted Similarity
def compute_combined_similarity(index, name_tfidf, artist_tfidf, name_weight, artist_weight):
    """Compute combined similarity for a given playlist index."""
    # Playlist name similarity
    name_similarity = cosine_similarity(name_tfidf[index], name_tfidf).flatten()
    # Artist name similarity
    artist_similarity = cosine_similarity(artist_tfidf[index], artist_tfidf).flatten()
    # Weighted combination
    combined_similarity = (name_weight * name_similarity) + (artist_weight * artist_similarity)
    return combined_similarity

# Step 6: Define Weights and Compute Similarity
name_weight = 0.5
artist_weight = 0.5
playlist_index = 31  # Example: Compute for playlist with ID 31

combined_similarity = compute_combined_similarity(
    playlist_index, name_tfidf, artist_tfidf, name_weight, artist_weight
)

# Step 7: Get Top Similar Playlists
top_indices = np.argsort(combined_similarity)[::-1][1:6]  # Exclude self (index 0)
top_playlist_ids = playlist_level_df.iloc[top_indices]['playlist_id'].tolist()

# Step 8: Output Results
print("Top 5 similar playlist IDs:", top_playlist_ids)

# Step 9: Display Playlist Titles
matching_playlists = playlist_level_df[playlist_level_df['playlist_id'].isin(top_playlist_ids)]
for _, row in matching_playlists.iterrows():
    print(f"Playlist ID: {row['playlist_id']}, Title: {row['playlist_name']}")

# Step 10: Recommend Tracks Not in Target Playlist
# Get tracks from the target playlist
target_tracks = set(final_df[final_df['playlist_id'] == playlist_index]['track_name'])

# Aggregate tracks from the top similar playlists
similar_tracks = final_df[final_df['playlist_id'].isin(top_playlist_ids)]['track_name']

# Filter tracks not in the target playlist
filtered_tracks = similar_tracks[~similar_tracks.isin(target_tracks)]

# Rank by frequency
track_recommendations = filtered_tracks.value_counts().head(10)

# Output recommended tracks
print(f"Recommended Tracks for Playlist ID {playlist_index} (not in original):")
for track_name, freq in track_recommendations.items():
    print(f"Track Name: {track_name}, Frequency: {freq}")


OSError: [E050] Can't find model 'en_core_web_md'. It doesn't seem to be a Python package or a valid path to a data directory.